In [1]:
import db
import models
import logging
import numpy as np
import pandas as pd
from tqdm import tqdm

from db import session
from models import *

from tokenizer import Tokenizer
from sklearn.decomposition import PCA
from collections import defaultdict

from pathlib import Path
from gensim.models import KeyedVectors

we = Path('/media/mquezada/HAHAHA/ams-data/phd/multimedia-summarization/data/word_embeddings/ft_alltweets_model.vec')
model = KeyedVectors.load_word2vec_format(we.as_posix())

tokenizer = Tokenizer()

pd.options.display.max_colwidth = 0

logger = logging.getLogger(__name__)
logging.basicConfig(format='%(asctime)s | %(name)s | %(levelname)s : %(message)s', level=logging.INFO)

2018-01-18 14:33:21,562 | summa.preprocessing.cleaner | INFO : 'pattern' package not found; tag filters are not available for English
2018-01-18 14:33:21,565 | gensim.models.keyedvectors | INFO : loading projection weights from /media/mquezada/HAHAHA/ams-data/phd/multimedia-summarization/data/word_embeddings/ft_alltweets_model.vec
2018-01-18 14:34:40,433 | gensim.models.keyedvectors | INFO : loaded (1076139, 100) matrix from /media/mquezada/HAHAHA/ams-data/phd/multimedia-summarization/data/word_embeddings/ft_alltweets_model.vec
2018-01-18 14:34:40,433 | tokenizer | INFO : Loading spacy model
2018-01-18 14:34:41,833 | tokenizer | INFO : Model Loaded


In [24]:
def gen_documents(id_tweet, urls_file, group_by_url=True):
    url_tweet = defaultdict(list)
    tweets_wo_url = list()

    filtered_tweets = []

    for tweet_id, (n_ht, n_urls) in tqdm(zip([t.tweet_id for t in tweets], 
                                             tokenizer.count_special_tokens([tweet.text for tweet in tweets])), 
                                         desc="filtering hashtags",
                                         total=len(tweets)):
        if n_ht < 4 and n_urls < 3:
            filtered_tweets.append(tweet_id)

    for tweet_id in tqdm(filtered_tweets, desc='extracting urls from tweets'):
        tweet = id_tweet[tweet_id]
        urls_ = [token.text for token in tokenizer.nlp(tweet.text) if token.like_url]
        for u in urls_:
            url_tweet[u].append(tweet.tweet_id)
        if not urls_:
            tweets_wo_url.append(tweet.tweet_id)

    exp_tweets = defaultdict(list)
            
    if urls_file:
        # 'data/urls.csv'
        filename = Path(urls_file)
        urls = pd.read_csv(filename.as_posix())
        short_exp = dict()

        for row in tqdm(urls.iterrows(), total=urls.shape[0], desc='loading urls data'):
            short_exp[row[1]['short_url']] = row[1]['expanded_clean']

        for short, expanded in tqdm(short_exp.items(), desc='expanded urls'):
            tweet_ids = url_tweet.get(short)
            if tweet_ids:
                exp_tweets[expanded] = tweet_ids
    else:
        urls = session.query(ShortURL, ExpandedURL)\
                .join(ExpandedURL, ShortURL.expanded_id == ExpandedURL.id).yield_per(1000)

    #for t in tweets_wo_url:
    #    exp_tweets[t] = [t]
        
    return exp_tweets

In [25]:
def gen_repr(exp_tweets, rep='fasttext'):
    vectors = []
    docs = []
    doc_text = []

    if rep == 'fasttext':
        for doc_id, tweet_ids in tqdm(exp_tweets.items(), total=len(exp_tweets), desc='fasttext vectors'):
            texts = [id_tweet[tweet_id].text for tweet_id in tweet_ids if tweet_id in id_tweet]
            vector = [model[token] for text in texts for token in tokenizer.tokenize(text) if token in model]

            if vector:
                vectors.append(np.mean(vector, axis=0))
                docs.append(doc_id)
                for text in texts:
                    doc_text.append((doc_id, text))
    
    return vectors, docs, doc_text

# Libya Hotel Attack

variables:

- tweets: tweet objs del evento
- id_tweet: dict de tweet_id => tweet obj
- url_tweet: dict de short_url => lista de tweet_id
- tweets_wo_url: lista de tweet_id sin URL
- short_exp: short_url => expanded_url
- exp_tweet: expanded_url => tweet_ids
- vectors: fasttext vectors for each expanded_url
- docs: expanded_urls
- doc_texts: list of pairs (expanded_url, text) for each text in tweets

In [27]:
base = Path('data/vectors_jan18/')

event_ids = (7, 9)  # 7, 8 ,9 

for event_id in tqdm(event_ids, desc='event'):
    tweets = db.get_tweets(event_id)
    logger.info(f'total {len(tweets)} tweets')
    id_tweet = dict()

    for tweet in tweets:
        id_tweet[tweet.tweet_id] = tweet

    exp_tweets = gen_documents(id_tweet=id_tweet, urls_file='data/urls.csv', group_by_url=True)
    logger.info(f'total {len(exp_tweets)} exp_tweets')
    vectors, docs, doc_text = gen_repr(exp_tweets, rep='fasttext')

    pd.DataFrame(vectors).to_csv((base / f'{event_id}_urls_fasttext_vectors.tsv').as_posix(), sep='\t', header=False, index=False)
    pd.DataFrame(docs).to_csv((base / f'{event_id}_urls_fasttext_docs.tsv').as_posix(), sep='\t', header=False, index=False)
    pd.DataFrame(doc_text).to_csv((base / f'{event_id}_urls_fasttext_doc_text.tsv').as_posix(), sep='\t', header=False, index=False)



event:   0%|          | 0/2 [00:00<?, ?it/s]2018-01-18 18:30:15,345 | __main__ | INFO : total 113215 tweets


filtering hashtags:   0%|          | 0/113215 [00:00<?, ?it/s]

filtering hashtags:   0%|          | 1/113215 [00:01<62:10:24,  1.98s/it]

filtering hashtags:   1%|          | 1001/113215 [00:03<06:56, 269.59it/s]

filtering hashtags:   2%|▏         | 2001/113215 [00:05<04:58, 372.28it/s]

filtering hashtags:   3%|▎         | 3001/113215 [00:07<04:21, 421.15it/s]

filtering hashtags:   4%|▎         | 4001/113215 [00:08<04:01, 451.49it/s]

filtering hashtags:   4%|▍         | 5001/113215 [00:10<03:49, 472.53it/s]

filtering hashtags:   5%|▌         | 6001/113215 [00:12<03:38, 490.76it/s]

filtering hashtags:   6%|▌         | 7001/113215 [00:13<03:31, 502.13it/s]

filtering hashtags:   7%|▋         | 8001/113215 [00:15<03:25, 511.18it/s]

filtering hashtags:   8%|▊         | 9001/113215 [00:17<03:23, 512.98it/s]

filtering hashtags:   9%|▉         | 10001/113215 [00:19<03:19, 51

filtering hashtags:  85%|████████▍ | 96001/113215 [02:47<00:30, 573.06it/s]

filtering hashtags:  86%|████████▌ | 97001/113215 [02:49<00:28, 573.12it/s]

filtering hashtags:  87%|████████▋ | 98001/113215 [02:51<00:26, 572.66it/s]

filtering hashtags:  87%|████████▋ | 99001/113215 [02:52<00:24, 572.75it/s]

filtering hashtags:  88%|████████▊ | 100001/113215 [02:54<00:23, 573.07it/s]

filtering hashtags:  89%|████████▉ | 101001/113215 [02:56<00:21, 572.90it/s]

filtering hashtags:  90%|█████████ | 102001/113215 [02:58<00:19, 572.52it/s]

filtering hashtags:  91%|█████████ | 103001/113215 [02:59<00:17, 572.35it/s]

filtering hashtags:  92%|█████████▏| 104001/113215 [03:01<00:16, 572.73it/s]

filtering hashtags:  93%|█████████▎| 105001/113215 [03:03<00:14, 573.46it/s]

filtering hashtags:  94%|█████████▎| 106001/113215 [03:04<00:12, 573.84it/s]

filtering hashtags:  95%|█████████▍| 107001/113215 [03:06<00:10, 574.04it/s]

filtering hashtags:  95%|█████████▌| 108001/113215 [03:08<00:09, 574

extracting urls from tweets:   1%|          | 1105/112231 [00:07<12:45, 145.16it/s]

extracting urls from tweets:   1%|          | 1121/112231 [00:07<12:44, 145.26it/s]

extracting urls from tweets:   1%|          | 1137/112231 [00:07<12:44, 145.37it/s]

extracting urls from tweets:   1%|          | 1153/112231 [00:07<12:43, 145.46it/s]

extracting urls from tweets:   1%|          | 1169/112231 [00:08<12:42, 145.60it/s]

extracting urls from tweets:   1%|          | 1185/112231 [00:08<12:42, 145.56it/s]

extracting urls from tweets:   1%|          | 1201/112231 [00:08<12:42, 145.64it/s]

extracting urls from tweets:   1%|          | 1217/112231 [00:08<12:41, 145.72it/s]

extracting urls from tweets:   1%|          | 1233/112231 [00:08<12:41, 145.79it/s]

extracting urls from tweets:   1%|          | 1249/112231 [00:08<12:40, 145.86it/s]

extracting urls from tweets:   1%|          | 1265/112231 [00:08<12:42, 145.59it/s]

extracting urls from tweets:   1%|          | 1280/112231 [00:08<

extracting urls from tweets:   2%|▏         | 2536/112231 [00:17<12:16, 148.85it/s]

extracting urls from tweets:   2%|▏         | 2551/112231 [00:17<12:17, 148.81it/s]

extracting urls from tweets:   2%|▏         | 2567/112231 [00:17<12:16, 148.84it/s]

extracting urls from tweets:   2%|▏         | 2583/112231 [00:17<12:16, 148.86it/s]

extracting urls from tweets:   2%|▏         | 2599/112231 [00:17<12:16, 148.83it/s]

extracting urls from tweets:   2%|▏         | 2614/112231 [00:17<12:16, 148.74it/s]

extracting urls from tweets:   2%|▏         | 2629/112231 [00:17<12:16, 148.75it/s]

extracting urls from tweets:   2%|▏         | 2645/112231 [00:17<12:16, 148.77it/s]

extracting urls from tweets:   2%|▏         | 2660/112231 [00:17<12:16, 148.75it/s]

extracting urls from tweets:   2%|▏         | 2675/112231 [00:17<12:16, 148.76it/s]

extracting urls from tweets:   2%|▏         | 2690/112231 [00:18<12:16, 148.75it/s]

extracting urls from tweets:   2%|▏         | 2705/112231 [00:18<

extracting urls from tweets:   4%|▎         | 3933/112231 [00:26<12:10, 148.21it/s]

extracting urls from tweets:   4%|▎         | 3949/112231 [00:26<12:10, 148.24it/s]

extracting urls from tweets:   4%|▎         | 3965/112231 [00:26<12:10, 148.24it/s]

extracting urls from tweets:   4%|▎         | 3981/112231 [00:26<12:10, 148.25it/s]

extracting urls from tweets:   4%|▎         | 3997/112231 [00:26<12:09, 148.27it/s]

extracting urls from tweets:   4%|▎         | 4013/112231 [00:27<12:09, 148.28it/s]

extracting urls from tweets:   4%|▎         | 4029/112231 [00:27<12:09, 148.27it/s]

extracting urls from tweets:   4%|▎         | 4045/112231 [00:27<12:09, 148.29it/s]

extracting urls from tweets:   4%|▎         | 4061/112231 [00:27<12:09, 148.28it/s]

extracting urls from tweets:   4%|▎         | 4077/112231 [00:27<12:09, 148.30it/s]

extracting urls from tweets:   4%|▎         | 4093/112231 [00:27<12:09, 148.31it/s]

extracting urls from tweets:   4%|▎         | 4109/112231 [00:27<

...

extracting urls from tweets:  24%|██▎       | 118793/503251 [12:29<40:26, 158.42it/s]

extracting urls from tweets:  24%|██▎       | 118810/503251 [12:29<40:26, 158.42it/s]

extracting urls from tweets:  24%|██▎       | 118826/503251 [12:30<40:26, 158.42it/s]

extracting urls from tweets:  24%|██▎       | 118843/503251 [12:30<40:26, 158.42it/s]

extracting urls from tweets:  24%|██▎       | 118860/503251 [12:30<40:26, 158.42it/s]

extracting urls from tweets:  24%|██▎       | 118876/503251 [12:30<40:26, 158.42it/s]

extracting urls from tweets:  24%|██▎       | 118893/503251 [12:30<40:26, 158.42it/s]

extracting urls from tweets:  24%|██▎       | 118910/503251 [12:30<40:26, 158.42it/s]

extracting urls from tweets:  24%|██▎       | 118927/503251 [12:30<40:26, 158.42it/s]

extracting urls from tweets:  24%|██▎       | 118943/503251 [12:30<40:25, 158.42it/s]

extracting urls from tweets:  24%|██▎       | 118960/503251 [12:30<40:25, 158.42it/s]

extracting urls from tweets:  24%|██▎      

extracting urls from tweets:  24%|██▍       | 120214/503251 [12:39<40:18, 158.35it/s]

extracting urls from tweets:  24%|██▍       | 120230/503251 [12:39<40:18, 158.35it/s]

extracting urls from tweets:  24%|██▍       | 120246/503251 [12:39<40:18, 158.35it/s]

extracting urls from tweets:  24%|██▍       | 120262/503251 [12:39<40:18, 158.35it/s]

extracting urls from tweets:  24%|██▍       | 120277/503251 [12:39<40:18, 158.34it/s]

extracting urls from tweets:  24%|██▍       | 120292/503251 [12:39<40:18, 158.34it/s]

extracting urls from tweets:  24%|██▍       | 120307/503251 [12:39<40:18, 158.34it/s]

extracting urls from tweets:  24%|██▍       | 120323/503251 [12:39<40:18, 158.34it/s]

extracting urls from tweets:  24%|██▍       | 120338/503251 [12:40<40:18, 158.33it/s]

extracting urls from tweets:  24%|██▍       | 120354/503251 [12:40<40:18, 158.33it/s]

extracting urls from tweets:  24%|██▍       | 120371/503251 [12:40<40:18, 158.33it/s]

extracting urls from tweets:  24%|██▍      

extracting urls from tweets:  24%|██▍       | 121620/503251 [12:48<40:11, 158.26it/s]

extracting urls from tweets:  24%|██▍       | 121636/503251 [12:48<40:11, 158.26it/s]

extracting urls from tweets:  24%|██▍       | 121652/503251 [12:48<40:11, 158.26it/s]

extracting urls from tweets:  24%|██▍       | 121669/503251 [12:48<40:11, 158.27it/s]

extracting urls from tweets:  24%|██▍       | 121685/503251 [12:48<40:10, 158.27it/s]

extracting urls from tweets:  24%|██▍       | 121702/503251 [12:48<40:10, 158.27it/s]

extracting urls from tweets:  24%|██▍       | 121718/503251 [12:49<40:10, 158.26it/s]

extracting urls from tweets:  24%|██▍       | 121734/503251 [12:49<40:10, 158.26it/s]

extracting urls from tweets:  24%|██▍       | 121750/503251 [12:49<40:10, 158.26it/s]

extracting urls from tweets:  24%|██▍       | 121766/503251 [12:49<40:10, 158.26it/s]

extracting urls from tweets:  24%|██▍       | 121782/503251 [12:49<40:10, 158.26it/s]

extracting urls from tweets:  24%|██▍      

KeyboardInterrupt: 


Exception in thread Thread-16:
Traceback (most recent call last):
  File "/home/mquezada/anaconda3/lib/python3.6/threading.py", line 916, in _bootstrap_inner
    self.run()
  File "/home/mquezada/anaconda3/lib/python3.6/site-packages/tqdm/_tqdm.py", line 144, in run
    for instance in self.tqdm_cls._instances:
  File "/home/mquezada/anaconda3/lib/python3.6/_weakrefset.py", line 60, in __iter__
    for itemref in self.data:
RuntimeError: Set changed size during iteration

